In [2]:
from toydaq import Motor, Diode, Camera, scan_iter


import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import *

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib import cm

from matplotlib import pyplot as plt
#from threading import Thread
from tqdm import tqdm
import numpy as np



# I use tkinter for creating the GUI and follow the MVC design patter to follow a serparate of concerns.
# Model: all hardware components inside and a future place for scan methods
# View: the GUI, visualises output, hardware. Talks to the user and the controller. Performs basic tasks like plotting and checks if parameter are selected
# Controller: Performs the workflows to bring motors and diodes, camera together. Sends events to functions
 
class Model():
    """ The model for our beamline. It contains our hardware, i.e. diodes, cameras, motors."""
    
    # Configuration of our beamline. Ex[ected parameters for the hardware.
    valid_units = ["m", "nm", "mm"]
    valid_diode_names=["INTENSITY", "COUNTER", "SIGNAL", "XRF"]
    valid_camera_names=["MY-CAMERA"]
    
    def __init__(self):
        """
        Make motors, diodes, cameras available in the model with their methods and attributes.
        better in the future: load from a config file
        """
        self.motors = {}
        self.diodes = {}
        self.cameras = {}

    def _check_valid_units(self, units):
        """
        Ensure the user has provided a valid unit name
        """
        if units not in self.valid_units:
            options = ", ".join(self.valid_units)
            raise ValueError(f"{units} is not a known unit type. Options are {options}")

    def add_motor(self, name, units):
        """
        Add a motor to the model
        """
        if name in self.motors:
            raise ValueError(
                f"Motor names must be unique: {name} is already a defined motor."
            )

        # Allow any casing
        units = units.lower()
        self._check_valid_units(units)
        print(f"Adding motor {name} with units {units}")
        self.motors[name] = Motor(name, units)

    def _check_valid_diode_names(self, diode_name):
        """
        Ensure the user has provided a valid diode name
        """   
        if diode_name not in self.valid_diode_names:
            options = ", ".join(self.valid_diode_names)
            raise ValueError(f"{diode_name} is not a known sensor diode name. Options are {options}")

    def add_diode(self, diode_name):
        """
        Add a diode to the model
        """
        if diode_name in self.diodes:
            raise ValueError(
                f"Diode names must be unique: {diode_name} is already a defined diode."
            )
        
        self._check_valid_diode_names(diode_name)
        print(f"Adding diode {diode_name}")
        self.diodes[diode_name]=Diode(diode_name)

    def _check_valid_camera_names(self, camera_name):
        """
        Ensure the user has provided a valid camera name
        """   
        if camera_name not in self.valid_camera_names:
            options = ", ".join(self.valid_camera_names)
            raise ValueError(f"{camera_name} is not a known sensor diode name. Options are {options}")


    def add_camera(self, camera_name):
        """Add a camera to the model"""

        if camera_name in self.cameras:
            raise ValueError(f"Camera names must be unique: {camera_name} is already a defined camera.")
        self._check_valid_camera_names(camera_name)
        print(f"Adding camera {camera_name}")
        self.cameras[camera_name]=Camera(camera_name)

  
    def diode_names(self):
        """ Return names of all available diodes """
        return [self.diodes[label].name for label in self.diodes]

    def motor_names(self):
        """ Return names of all available motors """
        return [self.motors[label].name for label in self.motors]

    def camera_names(self):
        """ Return names of all available cameras """
        return [self.cameras[label].name for label in self.cameras]
    
class View():
    # The view will be responsible for displaying widgets
    # and getting some information from user and from model.

    def __init__(self, model):
        """ Creates the main frame and links other frames to it. Only this layer should be visible to the model and controller. """

        self._model = model

        self._master = tk.Tk()
 
        self._frame = tk.Frame(self._master)
        self.fig = Figure(figsize=(7.5, 4), dpi=80)

        self.ax0 = self.fig.add_subplot()
        self._frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
        
        # The canvas is the drawing arrea
        self.canvas = FigureCanvasTkAgg(self.fig, master=self._frame)
        # Navigation toolbar for the canvas
        toolbar = NavigationToolbar2Tk(self.canvas, self._frame)
        toolbar.update()
        self.canvas.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=1)

        self.sidepanel = SidePanel(self._frame, self,  self._model)   #parent,  guicontrol, model as parame for SidePannel, this is how you establish communication :-)
        self.motorpanel= MotorPanel(self._frame, self, self._model)   #parent,  guicontrol, model as parame for Motorpanel 


        self.cbVariables={}
        self.cb={}
        #self.cbTexts={}
        #create checkboxes for diodes
        for diode_name in self._model.diode_names():
            self.cbVariables[diode_name]=tk.IntVar() #default value=0   #master=self.view.sidepanel.frame3
            #self.cbTexts[diode_name] = tk.StringVar()
            self.cb[diode_name]=ttk.Checkbutton(self.sidepanel.frame3,text=diode_name, variable=self.cbVariables[diode_name], state=['!alternate','!selected'], offvalue=0, onvalue=1, command=lambda diode_name=diode_name: self.on_cb_diode_selection(diode_name))
            self.cb[diode_name].pack(side=tk.TOP, anchor=tk.W) 
            print(f"Test on diode {diode_name} state at start: !alternate: {self.cb[diode_name].instate(['!alternate'])}, !selected: {self.cb[diode_name].instate(['!selected'])}") 


        self.cbVariables_cam={}
        self.cb_cam={}
        # create checkboxes for cameras
        for camera_name in self._model.camera_names():
            self.cbVariables_cam[camera_name]=tk.IntVar()
            self.cb_cam[camera_name]=ttk.Checkbutton(self.sidepanel.frame5,text=camera_name, variable=self.cbVariables_cam[camera_name], state=['!alternate','!selected'], offvalue=0, onvalue=1, command=lambda camera_name=camera_name: self.on_cb_cam_selection(camera_name))
            self.cb_cam[camera_name].pack(side=tk.TOP, anchor=tk.W) 
            print(f"Test on camera {camera_name} state at start: !alternate: {self.cb_cam[camera_name].instate(['!alternate'])}, !selected: {self.cb_cam[camera_name].instate(['!selected'])}") 


    # Methods 
           
    def on_create_scanplot(self, m):
        """ Connects the 1D scan plot button to a method """
        self.sidepanel.plotBut.bind("<Button>", m)

    def on_create_camplot(self, m):
        """ Connects the plot cam scan button to a method """
        self.sidepanel.plotCamBut.bind("<Button>", m)

    def on_clear(self, m):
        """ Clear canvas area """
        self.sidepanel.clearButton.bind("<Button>", m)

    def on_combobox_selection(self, m):  
        """ Connects combobox to a method """
        self.sidepanel.motor_selCombo.bind("<<ComboboxSelected>>", m)    

    def on_show_scan_settings(self,m):
        """ Connects the button show scan settings to a method """
        self.motorpanel.show_scan_settingsButton.bind("<Button>", m)

    def on_cb_diode_selection(self,diode_name): 
        """ Prints out the current state of a diode """
        print(f"Diode {diode_name} is {self.cb[diode_name].state()}")

    def on_cb_cam_selection(self,camera_name): 
        """ Prints out the current state of a camera """
        print(f"Camera {camera_name} is {self.cb_cam[camera_name].state()}")

        
    def run(self):
        """ Main method to start the GUI """
        self._master.geometry("1200x600")
        self._master.title("Tkinter MVC example")
        self._master.deiconify()
        self._master.mainloop()

    def on_motor_selection(self, mot_selected):
        """ Sends information about current motor to the label """
        # shows the selected motor and its position in the label
        #self.sidepanel.curr_motor_valLabel.configure(text=self._model.motors[mot_selected])
        self.sidepanel.curr_motor_val.set(self._model.motors[mot_selected])

        print(f"{mot_selected} was selected.")

        # adjust the units shown above the motor input values, when the motor is changed by the user 
        for column in self.motorpanel.motor_inputTexts:
            self.motorpanel.motor_inputTexts[column].set(f"{column} [{self._model.motors[mot_selected].units}]")
           
        # return motor as object 
        return self._model.motors[mot_selected]


    def update_current_motor_pos(self, mot_selected):
        """ Sends current motor information to the label """
        # The plan was to use this function to show a continuous update of the motor values. But I need threading. Not yet implemented.
        self.sidepanel.curr_motor_val.set(self._model.motors[mot_selected])

    def get_motor_selection(self):
        """ Get current motor selection from combobox and return motor """
        try:
            if self.sidepanel.motor_selCombo.get() == "":  
                raise ValueError('No motor selected')
            else:
                print('Current selected motor ', self.sidepanel.motor_selCombo.get())

                #returns only the motor name, i.e. str
                #return self.sidepanel.motor_selCombo.get()
                # returns a motor object
                return self._model.motors[self.sidepanel.motor_selCombo.get()]

        except ValueError as e:
            self.no_motor_selected()
            showwarning(title='Empty motor selection', message="Select motor.")
        
    def get_diode_selection(self):
        """ Returns which diodes is selected. Two choices: by state or by variable. I went with by variable."""
        #for label in self.cbVariables:
        #    print(label, self.cbVariables[label].get())

        for diode_name in self.cb:
            if self.cb[diode_name].instate(['selected']) == True:
                print(f"Diode {diode_name} is selected.")

        diodes_cb_values=[v.get() for v in self.cbVariables.values()]
        print(diodes_cb_values)

        if all([ w == 0 for w in diodes_cb_values]):
            self.no_diode_selected()
            return 
        else:
            all_selected_diodes={k:v for (k,v) in self.cbVariables.items() if v.get() == 1}
            print(all_selected_diodes, "type is", type(all_selected_diodes))
        # returns a dict of selected diodes
        return all_selected_diodes

    def get_camera_selection(self):
        """ Gets the current camera selection """
        cameras_cb_values=[v.get() for v in self.cbVariables_cam.values()]

        if all([ w == 0 for w in cameras_cb_values]):
            self.no_camera_selected()
            return 
        else:
            all_selected_cameras={k:v for (k,v) in self.cbVariables_cam.items() if v.get() == 1}
            print(all_selected_cameras, "type is", type(all_selected_cameras))

        # Our model beamline has only one camera curently. 
        if np.sum(cameras_cb_values) !=1:
            showwarning(title="Select one camera", message="Select one camera.")

        # returns a dict of selected cameras
        return all_selected_cameras
 

    def show_scan_settings(self): 
        """ Current motor, start, stop, stepsize is shown """
        curr_mot_selected=self.get_motor_selection().name  #str
        print(f"{curr_mot_selected} is selected.")
        # access the ttk.Entry 
        out_strings=[]
        for column in self.motorpanel.motor_inputTexts:
            print(print('Identify', self.motorpanel.entries[column]))
            # Label : Value
            out_str=f"{self.motorpanel.motor_inputTexts[column].get()}: {self.motorpanel.entriesValues[column].get()}"
            print(out_str)
            out_strings.append(out_str)

            # Both options possible, get only one, I go for the Variables of the ttk.Entry
            #print(f"This is the current Variable for {self.motorpanel.entries[column]}: {self.motorpanel.entriesValues[column].get()}")
            #print("This is the current value via.get on the ttk.Entry", self.motorpanel.entries[column].get())
        showinfo("Current scan settings", f"Motor: {curr_mot_selected}, {str(out_strings)[1:-1]}")
        
    def retrieve_scan_settings(self):
        """ Collect scan settings """

        mot_selected=self.get_motor_selection()
        mot_scan_settings=self.motorpanel.entriesValues

        return (mot_selected, mot_scan_settings)


    def line_colors(self, number_of_lines):
        """ Create better line colors """
        start = 0.0
        stop = 1.0
        cm_subsection = np.linspace(start, stop, number_of_lines)
        colors = [cm.jet(x) for x in cm_subsection]
        return colors
        
    def modify_1d_plot(self, mot, diodes):
        """ Modify title and xlabel for 1D scan with diodes """
        self.ax0.set_title('1D Scan')
        self.ax0.set_xlabel(f"{mot.name} [{mot.units}]")
    
    def plot_1d_scan(self, mot_selected, dio_selected, mot_values, dio_values):
        """ Plot 1D scan, one motor, multiple diodes """

        # some graphic modificaton
        self.modify_1d_plot(mot_selected, dio_selected)
        marker = ['o', 'v', '^', '<', '>', 's', '8', 'p']
        colors = ['b', 'g', 'r', 'c', 'm', 'k']
        handles=[]
        labels=[]
        for i in range(len(dio_selected)):
            line, =self.ax0.plot(mot_values, dio_values[i],  marker=marker[i], color=colors[i])
            #keep infos on legend.
            handles.append(line)
            labels.append(dio_selected[i])
        self.ax0.legend(handles,labels, loc="best")
        #this updates the plot (see Matplotlib doc)
        self.canvas.draw_idle()
        self.fig.canvas.flush_events()

    def plot_cam_1d_scan(self, mot_selected, cam_image, camera):
        """ Plot 1D scan with a camera, one motor, one camera """
        cam_im=self.ax0.imshow(cam_image, interpolation="none")
        self.ax0.set_title(f'1D Scan {camera}')
        self.ax0.set_xlabel(f"{camera} X")
        self.ax0.set_ylabel(f"{camera} Y")
        self.canvas.draw_idle()
        self.fig.canvas.flush_events()


    def no_motor_selected(self):
        """ Warning no motor selected """
        showwarning(title='1D scan warning', message="No motor selected for 1D scan.")
        
    
    def no_diode_selected(self):
        """ Warning no diode selected """
        showwarning(title='1D scan warning', message="No diode selected for 1D scan.")
        
    def no_camera_selected(self):
        """ Warning no camera selected """
        showwarning(title='1D scan warning', message="No camera selected for 1D scan.")

class SidePanel():
    """ This class contains the buttons, the combobox, the ouput label for the motor, label frames for diodes """
    def __init__(self, root, guicontrol, model):
        self.guicontrol=guicontrol
        self._model=model
        self._frame2 = tk.Frame(root)
        self._frame2.pack(side="top", fill=tk.BOTH, expand=1)
        self.plotBut = tk.Button(self._frame2, text="Plot 1D Scan")
        self.plotBut.pack(side="top", fill=tk.BOTH)
        self.plotCamBut = tk.Button(self._frame2, text="Plot Cam Scan")
        self.plotCamBut.pack(side="top", fill=tk.BOTH)
        self.clearButton = tk.Button(self._frame2, text="Clear")
        self.clearButton.pack(side="top", fill=tk.BOTH)


        # Create a frame
        self.motor_sel_title=ttk.Label(self._frame2, text='Motor selection').pack()
        motor_name=tk.StringVar() #Motornames are strings
        self.motor_selCombo=ttk.Combobox(self._frame2, textvariable=motor_name)
        self.motor_selCombo['values']=self._model.motor_names()
        self.motor_selCombo['state'] = 'readonly'
        #motor_sel.grid(column=0, row=1)
        self.motor_selCombo.pack(side="top", fill=tk.BOTH)
    

        #Create display for current motor values
        self.curr_motor_val=tk.StringVar()
        self.curr_motor_valLabel=ttk.Label(self._frame2, textvariable=self.curr_motor_val)
        self.curr_motor_valLabel.pack(side="top", fill=tk.BOTH)

         # Create LabelFrame for diodes
        self.frame3=tk.LabelFrame(self._frame2, height=150, text="Diodes")
        self.frame3.pack(side=tk.TOP, fill=tk.X, expand=0, anchor=tk.NW)
        self.frame3.config(highlightcolor='blue', highlightthickness=5, highlightbackground='blue', labelanchor="nw")

        #Create LabelFrame for camera
        self.frame5=tk.LabelFrame(self._frame2, height=150, text="Cameras")
        self.frame5.pack(side=tk.TOP, fill=tk.X, expand=0, anchor=tk.NW)
        self.frame5.config(highlightcolor='blue', highlightthickness=5, highlightbackground='blue', labelanchor="nw")

 
class MotorPanel():
    """ This class contains the show scan bettons and the entries for the motor positon.  #TODO: Boxes for soft limits for motos"""
    #You cannot use both pack and grid on widgets that have the same master.
    #SidePanel._frame2 is a child of MainWindow._master
    #MotorPanel.frame4 is a child of MainWindow._master too
    #both have same parent, both needs to use either pack or grid   
  
    def __init__(self, root, guicontrol, model):
        self.guicontrol=guicontrol
        self._model=model
        self.frame4=tk.Frame(root)
        self.frame4.pack(side='top',fill=tk.BOTH, expand=1) 
        
        self.entries = {}
        self.entriesValues = {}
        self.motor_input_label=["Start", "Stop", "Stepsize"]  #hard coded, but extendable for software limits
        self.motor_input_titleLabel={}
        self.motor_inputTexts ={}

        # Function to validate entries to the ttk.Entry 
        vcmd = (self.frame4.register(self.onValidate), '%P', '%s', '%S','%W')

        for i, column in enumerate(self.motor_input_label):
            self.motor_inputTexts[column] =tk.StringVar(value=column)
            #self.motor_inputTexts[column].set(f"{column}") #default labels without units
            self.motor_input_titleLabel[column] = ttk.Label(self.frame4, font=('Calibri', 13), textvariable=self.motor_inputTexts[column])
            self.motor_input_titleLabel[column].grid(row=0, column=i, sticky="nswe")
           

            self.entriesValues[column]= tk.DoubleVar()
            self.entries[column] = ttk.Entry(self.frame4, font=('Calibri', 13), textvariable=self.entriesValues[column], justify='left', validate="key", validatecommand=vcmd)
            self.entries[column].grid(row=1, column=i, sticky="nswe")

            
        #lets create a push button
        self.show_scan_settingsButton=ttk.Button(self.frame4,text="Show scan settings")
        self.show_scan_settingsButton.grid(row=2,columnspan=len(self.motor_input_label))


    def onValidate(self, P, s, S, W):
        """This should validate the input for each ttk.Entry
        %P = value of the entry if the edit is allowed
        %s = value of entry prior to editing
        %S = the text string being inserted or deleted, if any
        %W = the tk name of the widget
        """
        if P.strip() == "":
            # allow blank string
            return True
        elif P.strip() == "-":
            return True
        elif P.strip() == "+":
            return True
        try:
            float(P)
            return True
        except ValueError:
            print(f'float({P}) raised ValueError in {W}.')
            showwarning(title=f'ValueError in {W}', message=f"{P} is not a valid input.")
            return False

        # TODO
        # Create entry limits for software limits
      
        
    

class Controller():
    """" Controller part of the MVC model """
   
    def __init__(self, model, view):
        
        self.model = model
        self.view=view
          
        # Establish connection between buttons in View and controller
        self.view.on_create_scanplot(self.create_scan_plot)
        self.view.on_create_camplot(self.create_cam_plot)
        self.view.on_clear(self.clear)

        self.view.on_combobox_selection(self.motor_selection)

        self.view.on_show_scan_settings(self.show_scan_settings)

            
 
    def clear(self, event):
        """" Clears canvas on event action"""
        self.view.ax0.clear()
        self.view.canvas.draw()


    def create_cam_plot(self, event):
        """ 1D scan with a camera on event action """

        # get scan configuration
        scan_dict=self.prepare_1d_scan_settings()

        # get camera selected, dict
        cam_sel_dict=self.view.get_camera_selection()
        # get the keys
        cam_selected=[k for k,v in  cam_sel_dict.items()]
        num_dio_sel=len(cam_selected)
        print(f"Selected cameras for scan are {cam_selected}.")
        print(scan_dict, cam_sel_dict)

        #clear graph
        self.view.ax0.clear()
        self.view.canvas.draw()

        x_values=[]
        for pos in scan_iter(scan_dict["mot_sel"], scan_dict["Start"], scan_dict["Stop"], scan_dict["Stepsize"], cb=None, show_progress=True):
            
            #save motor positions
            x_values.append(pos)
            for i in cam_selected:
                img = self.model.cameras[i].get()
            self.view.plot_cam_1d_scan(mot_selected=scan_dict["mot_sel"], cam_image=img, camera=cam_selected)


    
    def prepare_1d_scan_settings(self):
        """ Retrieves relevant information for scan and prepapres scan dict for output (1D scan)"""

        mot_selected, scan_values=self.view.retrieve_scan_settings()

        #["Start", "Stop", "Stepsize"]
        start= scan_values["Start"].get()
        stop= scan_values["Stop"].get()
        stepsize= scan_values["Stepsize"].get()
       
        #check
        try:
            if stepsize > np.absolute(stop - start):
                raise ValueError("Stepszie larger than total scan range.")
            
        except ValueError as e:
            showwarning(title='Stepsize too large.', message="Stepszie larger than total scan range.")
            return

        try:
            if start > stop:
                raise ValueError("Start>Stop")
        except ValueError as e:
            showwarning(title="Start>Stop", message="Please select start < stop.")

        try:
            if stepsize < 0:
                raise ValueError("Stepsize negative.")

        except ValueError as e:
            showwarning(title="Stepsize negative." , message="Choose positive Stepsize.") 
        
        try:
            if stepsize==0:
                raise ValueError(f"Stepsize is {stepsize}")
        except ValueError as e:
            showwarning(title=f"Stepsize {stepsize}." , message="Choose different Stepsize.") 
        
        
        scan_dict={}
        scan_dict["mot_sel"]=mot_selected
        scan_dict["Start"]=start
        scan_dict["Stop"]=stop
        scan_dict["Stepsize"]=stepsize
        return scan_dict

    def create_scan_plot(self, event):
        """ Procedure for the 1D scan plot with diodes """       
        #get the diodes
        dio_selected_dict=self.view.get_diode_selection()

        # get the keys
        dio_selected=[k for k,v in  dio_selected_dict.items()]
        num_dio_sel=len(dio_selected)
        print(f"Selected diodes for scan are {dio_selected}.")
       
        # get scan configuration
        scan_dict=self.prepare_1d_scan_settings()

        #clear graph
        self.view.ax0.clear()
        self.view.canvas.draw()


        # prepare for plot
        #self.view.modify_1d_plot(self.model.motors[mot_selected], dio_selected)
        #self.view.modify_1d_plot(mot_selected, dio_selected)
                  
        x_values=[]
        diode_values = [[] for i in range(0, num_dio_sel)]

        # Pseudo exectuition of motor
        #for pos in scan_iter(self.model.motors[mot_selected], start, stop, stepsize, cb=None, show_progress=False):
        for pos in scan_iter(scan_dict["mot_sel"], scan_dict["Start"], scan_dict["Stop"], scan_dict["Stepsize"], cb=None, show_progress=True):
            
            #save motor positions
            x_values.append(pos)
            # read out and save diode positions
            for i in range(num_dio_sel):
                diode_values[i].append(self.model.diodes[dio_selected[i]].get())
                #print(dio_selected[i])
                #print(self.model.diodes[dio_selected[i]])

            # this is the function that sends data for plotting to view
            self.view.plot_1d_scan(mot_selected=scan_dict["mot_sel"],dio_selected=dio_selected, mot_values=x_values, dio_values=diode_values)
            
            #nope this does not work?
            #update motor position on GUI:
            #self.view.update_current_motor_pos(scan_dict["mot_sel"])
            # I think this is where you need threading

        #TODO: 
        # move motore per step and read out at every step
        # if I had better access to the scan function, this whole function would be nicer 


    def motor_selection(self, event):
        """ Selects motors on event action """
        # this method is directly bound to the function, so we can make use of this option
        mot_selected = event.widget.get()
        #show the current motor value after selection in the GUI
        self.view.on_motor_selection(mot_selected)


    def show_scan_settings(self, event):
        """ Show scan settings on event action"""
        self.view.show_scan_settings()


def main():
    """ Setting the stage for the GUI action """
    # create model and add diodes, motors, camera
    # TODO: hide adding of motors in Model?
    model = Model()
    motors = {"MY-LARGE-MOTOR": "m", "MY-TINY-MOTOR": "nm", "MY-NORMAL-MOTOR": "mm"}
    for motor_name, units in motors.items():
        model.add_motor(motor_name, units)
   
    diodes = {"INTENSITY": None, "COUNTER": None, "SIGNAL": None, "XRF": None} 
    for diode_name in diodes.keys():
        model.add_diode(diode_name) 

    cameras = {"MY-CAMERA": None}
    for camera_name in cameras.keys():
        model.add_camera(camera_name)

    # prepare GUI
    view = View(model)
   

    # inject into Controller
    c = Controller(model, view)
   
    # start the GUI
    view.run()


if __name__ == '__main__':
    main()
  

Adding motor MY-LARGE-MOTOR with units m
Adding motor MY-TINY-MOTOR with units nm
Adding motor MY-NORMAL-MOTOR with units mm
Adding diode INTENSITY
Adding diode COUNTER
Adding diode SIGNAL
Adding diode XRF
Adding camera MY-CAMERA
0 Start
1 Stop
2 Stepsize
Test on diode INTENSITY state at start: !alternate: True, !selected: True
Test on diode COUNTER state at start: !alternate: True, !selected: True
Test on diode SIGNAL state at start: !alternate: True, !selected: True
Test on diode XRF state at start: !alternate: True, !selected: True
Test on camera MY-CAMERA state at start: !alternate: True, !selected: True


In [2]:
from toydaq import Motor, Diode, Camera, scan_iter
from tqdm import tqdm


mot1=Motor("mot1").

SyntaxError: invalid syntax (1092754772.py, line 5)